In [69]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Assistant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Assistant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Assistant\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Assistant\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [70]:
df_train = pd.read_csv("Resources/twitter_training.csv", header=None)
df_train.head()

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [71]:
df_test = pd.read_csv("Resources/twitter_validation.csv",header=None)
df_test.head()

,0,1,2,3
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [72]:
df_train = df_train.drop([0,1],axis=1)

In [73]:
df_train = df_train.rename(columns={3:'text'})
df_train = df_train.rename(columns={2:'sentiment'})

,sentiment,text
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [80]:
df_train['text'].isna().sum()

686

In [81]:
df_train.dropna(subset=['text'],inplace=True)

In [92]:
print('text n/a: ' + str(df_train['text'].isna().sum()))
print('sentiment n/a: ' + str(df_train['sentiment'].isna().sum()))

text n/a: 0
sentiment n/a: 0


In [74]:
df_test = df_test.drop([0,1],axis=1)
df_test = df_test.rename(columns={3:'text'})
df_test = df_test.rename(columns={2:'sentiment'})

In [93]:
print('text n/a: ' + str(df_test['text'].isna().sum()))
print('sentiment n/a: ' + str(df_test['sentiment'].isna().sum()))

text n/a: 0
sentiment n/a: 0


In [94]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    
    # Tokenization (split the text into words)
    tokens = nltk.word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization (reduce words to their base form)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join the tokens back into a single string
    text = ' '.join(tokens)
    
    return text

# Apply the text pre-processing function to the '3' column
df_train['text'] = df_train['text'].apply(preprocess_text)


In [97]:
df_test['text'] = df_test['text'].apply(preprocess_text)

In [98]:
df_train = pd.get_dummies(df_train, columns=['sentiment'], prefix='sentiment')

In [99]:
df_train.head()

,text,sentiment_Irrelevant,sentiment_Negative,sentiment_Neutral,sentiment_Positive
0,im getting borderland murder,0,0,0,1
1,coming border kill,0,0,0,1
2,im getting borderland kill,0,0,0,1
3,im coming borderland murder,0,0,0,1
4,im getting borderland 2 murder,0,0,0,1
